# Processing LAPD bookings received via email

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import geojson
import json
import jenkspy
import numpy as np
from altair import datum
import altair as alt
alt.renderers.enable('notebook')
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
pd.options.display.float_format = '{:.0f}'.format

In [2]:
# read csv we've exported from postgres database with bookings
src = pd.read_csv('/Users/mhustiles/data/data/LA/latimes_bookings.csv',low_memory=False)

In [3]:
src.columns = src.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [4]:
src = src.astype({'id':'object', 
            'attachment_id':'object', 
            'booking_num':'object', 
            'jail':'object', 
            'arrest_zipcode':'object'})

In [5]:
src.drop_duplicates(inplace=True)

In [6]:
bookings = pd.DataFrame(src)

In [7]:
bookings['nu_booking_date'] = bookings['booking_date']\
    .str.replace(':00-08', ':00', regex=False).str.replace(':00-07', ':00', regex=False)

bookings['nu_booking_date'] = pd.to_datetime(bookings['nu_booking_date'], format='%Y-%m-%d %H:%M:%S')

In [8]:
bookings['nu_arrest_date'] = bookings['arrest_date']\
    .str.replace(':00-08', ':00', regex=False).str.replace(':00-07', ':00', regex=False)

bookings['nu_arrest_date'] = pd.to_datetime(bookings['nu_arrest_date'], format='%Y-%m-%d %H:%M:%S')

In [9]:
bookings['nu_release_date'] = bookings['release_date']\
    .str.replace(':00-08', ':00', regex=False).str.replace(':00-07', ':00', regex=False)

bookings['nu_release_date'] = pd.to_datetime(bookings['nu_release_date'], format='%Y-%m-%d %H:%M:%S')

In [10]:
bookings['arrest_year'] = bookings['nu_arrest_date'].dt.year
bookings['arrest_day'] = bookings['nu_arrest_date'].dt.day
bookings['arrest_month'] = bookings['nu_arrest_date'].dt.month
bookings['arrest_weekday'] = bookings['nu_arrest_date'].dt.weekday_name
bookings['arrest_monthname'] = bookings['nu_arrest_date'].dt.month_name()
bookings['arrest_hour'] = bookings['nu_arrest_date'].dt.hour.astype(object)

In [11]:
bookings.dtypes

id                             object
attachment_id                  object
booking_num                    object
booking_date                   object
jail                           object
first_name                     object
middle_name                    object
last_name                      object
suffix                         object
gender                         object
race                           object
hair_color                     object
eye_color                      object
height                        float64
weight                        float64
date_of_birth                  object
occupation                     object
arrestee_address               object
arrestee_city                  object
arrestee_state                 object
arrest_date                    object
arrest_address                 object
arrest_zipcode                 object
charge_string                  object
total_bail                    float64
no_bail                        object
release_date

---

In [12]:
bookings.arrestee_address = bookings.arrestee_address.fillna('')
bookings.booking_num = bookings.booking_num.astype(str)

In [13]:
bookings['homeless'] = bookings['arrestee_address'].str.contains('1942 TRANSIENT')

In [ ]:
booleanDict = {True: 'Homeless', False: 'Not homeless'}
bookings = bookings.replace(booleanDict)

In [ ]:
bookings = bookings.fillna('')

In [ ]:
bookings.loc[0]

---

## Arrests trends for homeless and non-homeless LA residents

### Count arrests by year and type. Drop '09, '20 & and handful of missing years

In [ ]:
years = bookings.groupby(['homeless', 'arrest_year'])\
    .agg('size').reset_index(name='bookings').drop([0,1,12,13,14,15,26,27])

In [ ]:
years.head(20)

### What's the homeless resident arrest share — and how has that changed over time?

In [ ]:
homeless_arrests_share = alt.Chart(years).mark_area().encode(
    y=alt.Y('sum(bookings):Q', stack="normalize", axis=alt.Axis(format='%', tickCount=7), title=' '),
    x=alt.X('arrest_year:N', title=' '),
    color=alt.Color('homeless', title=' '),
    order=alt.Order('sum(bookings):Q', sort='ascending')
).properties(width=600,height=300, title='Homeless share of LAPD arrests')

homeless_arrests_share.configure_legend(
    orient='top'
)

### How have LAPD's homeless arrest counts changed overtime?

In [ ]:
homeless_arrests_trend_line = alt.Chart(years).mark_line().encode(
    x=alt.X('arrest_year:N', title='Year'),
    y=alt.Y('bookings:Q', axis=alt.Axis(tickCount=6, format=''), title=' '),
    color=alt.Color('homeless', title=' ')
)

homeless_arrests_trend_line.configure_legend(
    orient='top',
    symbolType='stroke',
    symbolSize=200
).properties(width=400,height=300, title='Annual LAPD arrests, by resident type')

### Pivot the years table for exort

In [ ]:
years_pivot = pd.DataFrame(years.pivot(index='arrest_year', \
                                       columns='homeless', values='bookings').reset_index())

In [ ]:
years_pivot = pd.pivot_table(years, index='arrest_year', 
                       columns=['homeless'],
                       values = 'bookings',
                       aggfunc=np.sum,
                       margins=True).reset_index().drop(10)

In [ ]:
years_pivot.rename(columns={"arrest_year": "year", 'Homeless':'homeless', \
                            'Not homeless':'not_homeless', "All": "total"})

In [ ]:
years_pivot.to_csv('output/years_pivot.csv')

---

## Import Los Angeles geography

### LAPD divisions

In [ ]:
divisions_src = gpd.read_file('/Users/mhustiles/data/github/notebooks/lapd-crimes-arrests/output/lapd_divisions.json')

In [ ]:
divisions_src.plot()

In [ ]:
divisions_src.columns = divisions_src.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [ ]:
divisions = divisions_src[['aprec', 'prec', 'area', 'geometry']]

### LA Times neighborhoods

In [ ]:
hoodssrc = gpd.read_file('/Users/mhustiles/data/github/notebooks/lapd-crimes-arrests/input/custom-la-hoods-skid-row.geojson')

In [ ]:
hoodssrc.columns = hoodssrc.columns.str.strip().str.lower().str.replace(' ', '_')\
    .str.replace('(', '').str.replace(')', '')

### Trim the neighborhoods dataframe

In [ ]:
hoods = hoodssrc[['name', 'geometry']]

In [ ]:
hoodssrc.plot()